## Thinkful exercise 3.2.5 - Dimensionality reduction using random forest model

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Project description

Data:  2015 data from Lending Club, found at this link:https://www.lendingclub.com/info/download-data.action

Modelling objective: Predict the state of a loan given some information about it

Assignment details: 

1) Get rid of as much data as possible without dropping below an average of 90% accuracy in a 10-fold cross validation.

2) Identify which features are most important. This can be the raw features or the generated dummies. You may want to use PCA or correlation matrices.

3) Can you [answer the question] without using anything related to payment amount or outstanding principal? How do you know?


### Preprocessing data

In [62]:
#Using nrows to restrict dataset size for development
y2015 = pd.read_csv(
    '../../Datafiles/unit_3/LoanStats3d.csv',
    skipinitialspace=True,
    header=1,
    low_memory = False
)
y2015 = y2015.sample(frac=0.20).reset_index(drop=True)

In [63]:
y2015.shape

(84219, 111)

#### The data dictionary includes 152 columns. This is a subset of the columns in the full dataset, apparently.

In [64]:
y2015.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,67695770,72549492.0,10000.0,10000.0,10000.0,36 months,9.76%,321.55,B,B3,...,0.0,1.0,100.0,100.0,0.0,0.0,28400.0,15509.0,11000.0,0.0
1,68615683,73518479.0,5000.0,5000.0,5000.0,36 months,10.78%,163.18,B,B4,...,0.0,0.0,100.0,100.0,0.0,0.0,103020.0,92494.0,15500.0,81520.0
2,40979213,43854930.0,5000.0,5000.0,5000.0,36 months,7.89%,156.43,A,A5,...,0.0,2.0,92.3,100.0,0.0,0.0,306523.0,21454.0,19000.0,9223.0
3,59159919,63047652.0,28350.0,28350.0,28250.0,36 months,7.89%,886.95,A,A5,...,0.0,2.0,100.0,100.0,0.0,0.0,366098.0,130136.0,33700.0,110460.0
4,42504081,45470839.0,12650.0,12650.0,12625.0,60 months,18.25%,322.95,E,E1,...,0.0,4.0,82.6,50.0,0.0,0.0,98808.0,27290.0,4400.0,29546.0


In [65]:
y2015.tail()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
84214,63506237,67849018.0,3200.0,3200.0,3200.0,36 months,12.29%,106.73,C,C1,...,0.0,7.0,77.3,75.0,0.0,0.0,56832.0,41557.0,17100.0,34632.0
84215,59099119,62986831.0,19525.0,19525.0,19525.0,60 months,17.86%,494.33,D,D5,...,0.0,2.0,80.0,100.0,0.0,0.0,361000.0,9589.0,7700.0,0.0
84216,58623173,62463921.0,11000.0,11000.0,11000.0,60 months,12.69%,248.55,C,C2,...,0.0,1.0,90.0,20.0,1.0,0.0,61381.0,44647.0,21700.0,38281.0
84217,50526731,53906503.0,28000.0,28000.0,28000.0,36 months,5.32%,843.22,A,A1,...,0.0,0.0,93.7,0.0,0.0,0.0,286091.0,102778.0,38200.0,108908.0
84218,44439504,47487234.0,12000.0,12000.0,12000.0,60 months,12.29%,268.70,C,C1,...,0.0,1.0,83.3,50.0,0.0,0.0,478944.0,138917.0,21700.0,136235.0


In [66]:
#remove two summary rows at the end that don't actually contain data.
y2015 = y2015[:-2]

In [67]:
y2015.shape

(84217, 111)

#### Analyze columns with null values and adjust contents to zero or drop them

In [68]:
#Compute proportion of null values
null_counts = pd.DataFrame(y2015.isna().mean().round(4) * 100, columns = ['null_pcnt'])
null_counts.index.name = 'col_name'
null_counts = null_counts.reset_index()
print(null_counts.loc[null_counts.null_pcnt > 0])

                           col_name  null_pcnt
10                        emp_title       5.66
11                       emp_length       5.64
19                             desc      99.99
21                            title       0.03
28           mths_since_last_delinq      48.40
29           mths_since_last_record      82.25
33                       revol_util       0.04
45                     last_pymnt_d       0.06
47                     next_pymnt_d      27.86
50      mths_since_last_major_derog      70.95
53                 annual_inc_joint      99.87
54                        dti_joint      99.87
55        verification_status_joint      99.87
59                      open_acc_6m      95.00
60                       open_il_6m      95.00
61                      open_il_12m      95.00
62                      open_il_24m      95.00
63               mths_since_rcnt_il      95.14
64                     total_bal_il      95.00
65                          il_util      95.63
66           

In [69]:
#Fill nan with zero in some columns, because that is what is implied for these columns given knowledge of the data
#Pandas 'startswith' works differently than Python. Python allows a beg and end argument
#num_tl prefix means 'number of accounts'
col_list = y2015.loc[:, y2015.columns.str.startswith('mths_')].columns
col_list = col_list.append(y2015.loc[:, y2015.columns.str.startswith('num_tl_')].columns)
col_list = col_list.append(y2015.loc[:, y2015.columns.str.startswith('mo_sin_')].columns)
col_list = col_list.append(y2015.loc[:, y2015.columns.str.contains('bc_')].columns)
for col in col_list:
    y2015[[col]] = y2015[[col]].fillna(value = 0)

In [70]:
#Review remaining columns with null values 
null_counts = pd.DataFrame(y2015.isna().mean().round(4) * 100, columns = ['null_pcnt'])
null_counts.index.name = 'col_name'
null_counts = null_counts.reset_index()
print(null_counts.loc[null_counts.null_pcnt > 0])

                     col_name  null_pcnt
10                  emp_title       5.66
11                 emp_length       5.64
19                       desc      99.99
21                      title       0.03
33                 revol_util       0.04
45               last_pymnt_d       0.06
47               next_pymnt_d      27.86
53           annual_inc_joint      99.87
54                  dti_joint      99.87
55  verification_status_joint      99.87
59                open_acc_6m      95.00
60                 open_il_6m      95.00
61                open_il_12m      95.00
62                open_il_24m      95.00
64               total_bal_il      95.00
65                    il_util      95.63
66                open_rv_12m      95.00
67                open_rv_24m      95.00
68                 max_bal_bc      95.00
69                   all_util      95.00
71                     inq_fi      95.00
72                total_cu_tl      95.00
73               inq_last_12m      95.00


In [71]:
# Convert interest rate to numeric.
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

In [72]:
#Identify columns that are not numeric and examine how variable their values are
categorical = y2015.select_dtypes(include=['object'])
for i in categorical:
    column = categorical[i]
    print(i)
    print(column.nunique())

id
84217
term
2
grade
7
sub_grade
35
emp_title
32852
emp_length
11
home_ownership
4
verification_status
3
issue_d
12
loan_status
7
pymnt_plan
1
url
84217
desc
5
purpose
13
title
14
zip_code
881
addr_state
49
earliest_cr_line
619
revol_util
1105
initial_list_status
2
last_pymnt_d
25
next_pymnt_d
3
last_credit_pull_d
26
application_type
2
verification_status_joint
3


In [73]:
# Drop columns with many unique variables
drop_cols_list = ['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'id', 'title', 'last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d']

In [74]:
y2015 = y2015.drop(columns = drop_cols_list)

In [75]:
#Drop any remaining columns with a high number of Nulls
y2015 = y2015.dropna(axis = 1)

### Thinkful baseline model

Thinkful commentary:

The score cross validation reports is the accuracy of the tree. Here we're about 98% accurate.

That works pretty well, but there are a few potential problems. Firstly, we didn't really do much in the way of feature selection or model refinement. As such there are a lot of features in there that we don't really need. Some of them are actually quite impressively useless.

There's also some variance in the scores. The fact that one gave us only 93% accuracy while others gave higher than 98 is concerning. This variance could be corrected by increasing the number of estimators. That will make it take even longer to run, however, and it is already quite slow.

### Robin's modeling

#### Examine the target variable

In [76]:
y2015.shape

(84217, 81)

In [77]:
y2015.loan_status.value_counts()

Current               57352
Fully Paid            17703
Charged Off            5761
Late (31-120 days)     1972
In Grace Period         879
Late (16-30 days)       389
Default                 161
Name: loan_status, dtype: int64

Note that the cut-off where delinquency matters is 16 days and more. In reality, it is likely one month and more than one month that is most meaningful.

#### Build some features with meaningful collections & delinquency information. The recent information is the most important.

Collections data and charge off data: It's most material to know whether there were collections or charge offs in the last 12 months.

Delinquency data: If months since last delinq = 0, means > = 0, < 30 days. So if zero, set no_delinq = True,  else False.

Separate months since last delinq into recent delinquencies -- delinq_last_12_mo = True if >0, <= 12 else false

Then drop cols in sel_cols list

In [78]:
#Change the recent delinquency parameter in number of months
recent_delinq_param = 12
#Use apply to set no_delinq, recent_delinq flags
y2015['no_delinq'] = np.where(y2015.mths_since_last_delinq == 0, True, False)
y2015['recent_delinq'] = np.where(((y2015.mths_since_last_delinq > 0) & (y2015.mths_since_last_delinq <= recent_delinq_param)), True, False)

In [79]:
y2015['no_derog'] = np.where(y2015.mths_since_last_major_derog == 0, True, False)
y2015['recent_derog'] = np.where(((y2015.mths_since_last_major_derog > 0) & (y2015.mths_since_last_major_derog <= 12)), True, False)
y2015['collections_12mo'] = np.where(y2015.collections_12_mths_ex_med == 0, True, False)
y2015['chargeoffs_12mo'] = np.where(((y2015.chargeoff_within_12_mths == 0)), True, False)

In [80]:
# Drop columns used to categorize delinquencies, chargeoffs, derogitories
drop_cols_list = ['mths_since_last_major_derog', 'mths_since_last_delinq', 
                  'collections_12_mths_ex_med', 'chargeoff_within_12_mths',
                  'out_prncp_inv']

In [81]:
y2015 = y2015.drop(columns = drop_cols_list)

In [82]:
assert pd.notnull(y2015).all().all()

#### Step 1. Normalize variance and drop columns with low variance

In [83]:
y2015.shape

(84217, 82)

In [84]:
#Set the target variable and drop it from the main dataset
y = y2015.loan_status
y2015 = y2015.drop('loan_status', axis = 1)

In [85]:
#Make a dataframe containing a subset of the columns - numeric only
num_y2015 = pd.DataFrame(y2015.select_dtypes(include=['float64', 'int64']))

In [86]:
from sklearn.feature_selection import VarianceThreshold 

var_thresh = VarianceThreshold(threshold=0.005)
var_thresh.fit(num_y2015 / num_y2015.mean())

#Here I am using a mask to reduce the features. I could have used fit_transform to do everything in one step
mask = var_thresh.get_support()
reduced_num_y2015 = num_y2015.loc[:, mask] 
print(reduced_num_y2015.shape)

(84217, 65)


17 features were dropped as low-variance (above)

#### Step 2. Encode the categorical variables that remain and merge the dataframe with the reduced set of numeric features

In [87]:
y2015 = pd.get_dummies(y2015)

In [88]:
#Merge the dataframe with reduced number of columns with the dataframe that includes encoded categorical variables
keep_cols = y2015.columns.difference(reduced_num_y2015.columns)
print(keep_cols)
y2015_new = pd.merge(reduced_num_y2015, y2015[keep_cols], left_index=True, right_index=True, how='outer')

Index(['application_type_INDIVIDUAL', 'application_type_JOINT',
       'chargeoffs_12mo', 'collections_12mo', 'grade_A', 'grade_B', 'grade_C',
       'grade_D', 'grade_E', 'grade_F', 'grade_G', 'home_ownership_ANY',
       'home_ownership_MORTGAGE', 'home_ownership_OWN', 'home_ownership_RENT',
       'initial_list_status_f', 'initial_list_status_w', 'issue_d_Apr-2015',
       'issue_d_Aug-2015', 'issue_d_Dec-2015', 'issue_d_Feb-2015',
       'issue_d_Jan-2015', 'issue_d_Jul-2015', 'issue_d_Jun-2015',
       'issue_d_Mar-2015', 'issue_d_May-2015', 'issue_d_Nov-2015',
       'issue_d_Oct-2015', 'issue_d_Sep-2015', 'no_delinq', 'no_derog',
       'policy_code', 'purpose_car', 'purpose_credit_card',
       'purpose_debt_consolidation', 'purpose_home_improvement',
       'purpose_house', 'purpose_major_purchase', 'purpose_medical',
       'purpose_moving', 'purpose_other', 'purpose_renewable_energy',
       'purpose_small_business', 'purpose_vacation', 'purpose_wedding',
       'pymnt_plan_

In [89]:
y2015_new.shape

(84217, 118)

In [91]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

X = y2015_new

rfc = ensemble.RandomForestClassifier()
rfc.fit(X, y)
cross_val_score(rfc, X, y, cv=10)

/Users/robin/anaconda3/envs/data_sci/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


array([ 0.95941135,  0.95726496,  0.95927333,  0.96009975,  0.95819974,
        0.95962475,  0.95819974,  0.95914974,  0.959506  ,  0.95961516])

#### I've achieved a score above 90%, but how many features are really necessary?

In [92]:
# Calculate the correlation matrix and take the absolute value
corr_matrix = y2015_new.corr().abs()

# Create a True/False mask and apply it
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
y2015_corr = corr_matrix.mask(mask)


# List column names of highly correlated features (r > 0.85). Changing the r value to 0.80 drops the score to around 90,
# so I'll stick with the value 0.85.
to_drop = [c for c in y2015_corr.columns if any(y2015_corr[c] >  0.85)]
# Drop the features in the to_drop list
y2015_reduced = y2015_new.drop(to_drop, axis=1)

print("The reduced dataframe has {} columns.".format(y2015_reduced.shape[1]))

The reduced dataframe has 105 columns.


In [93]:
#These columns will be dropped because they are highly correlated with others
print(to_drop)

['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'open_acc', 'total_pymnt', 'total_pymnt_inv', 'recoveries', 'tot_cur_bal', 'num_actv_rev_tl', 'total_bal_ex_mort', 'application_type_INDIVIDUAL', 'initial_list_status_f', 'term_ 36 months']


In [94]:
X = y2015_reduced
rfc.fit(X, y)
cross_val_score(rfc, X, y, cv=10)

array([ 0.9566817 ,  0.9578585 ,  0.95571123,  0.95784349,  0.95546847,
        0.95724973,  0.95641848,  0.95629973,  0.95689348,  0.95640812])

In [96]:
feature_importances = pd.DataFrame(rfc.feature_importances_,
                                   index = X.columns,
                                    columns=['importance']).sort_values('importance',    
                                    ascending=False
                                    )

In [101]:
feature_importances['cum_importance'] = feature_importances.cumsum(axis = 0)
print(feature_importances.head(40))

                                importance  cum_importance
last_pymnt_amnt                   0.312681        0.312681
out_prncp                         0.239921        0.552601
total_rec_prncp                   0.126136        0.678738
total_rec_int                     0.042234        0.720972
collection_recovery_fee           0.036636        0.757608
installment                       0.026601        0.784209
member_id                         0.012629        0.796838
int_rate                          0.010164        0.807001
annual_inc                        0.006693        0.813695
dti                               0.006468        0.820162
revol_bal                         0.006267        0.826429
mo_sin_old_rev_tl_op              0.006194        0.832623
tot_hi_cred_lim                   0.006137        0.838760
bc_util                           0.006100        0.844860
total_bc_limit                    0.006089        0.850949
bc_open_to_buy                    0.005945        0.8568

#### Using PCA degrades the accuracy score (from 95% accuracy to about 86%) while using 40 features. PCA doesn't perform better than just taking into account the feature importance measure and establishing a cutoff when the accuracy score reaches an acceptable level.

In [33]:
from sklearn.preprocessing import StandardScaler 
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

pipe = Pipeline([
('scaler', StandardScaler()),
('reducer', PCA(n_components=40)), 
('classifier', ensemble.RandomForestClassifier())])

pipe.fit(X, y)

print(cross_val_score(pipe, X, y, cv = 10))

/Users/robin/anaconda3/envs/data_sci/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype bool, uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/robin/anaconda3/envs/data_sci/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype bool, uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/Users/robin/anaconda3/envs/data_sci/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/robin/anaconda3/envs/data_sci/lib/python3.6/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 6 members, which is too few. The minimum number of members in any class cannot be

[ 0.86238532  0.86217458  0.86328725  0.86021505  0.86175115  0.84
  0.86244204  0.86862442  0.84080371  0.87326121]


/Users/robin/anaconda3/envs/data_sci/lib/python3.6/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype bool, uint8, int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


### Discussion

1) Get rid of as much data as possible without dropping below an average of 90% accuracy in a 10-fold cross validation.

My random forest model gets above a 90% accuracy score with 25 specific features; the model achieves ~95% accuracy with about 40 features. None of the generated dummies make the cut, so if this model were to go into production, we could skip that step and work with a smaller dataset from the outset.

The additional features I created also did not contribute to explanatory power in a significant way.

2) Identify which features are most important. This can be the raw features or the generated dummies. You may want to use PCA or correlation matrices.

The most important features are outstanding principal and last payment amount. Together they explain almost 70% of the variability in the dataset.

3) Can you [answer the question] without using anything related to payment amount or outstanding principal? How do you know?

Since outstanding principal and last payment amount account for almost 70% of the variability in the dataset, it  would not be possible to answer the question accurately if one excluded payment amount or outstanding principal.

[Note: Outstanding principal column names are 'out_princp' and 'out_princp_inv'). Payment amount column name is 'installment'.]

One question I am left with is, Why does a value like "member id" have any explanatory power at all? Is it because it may go from low to high values and thereby give some information about how recent the loan is?